**import libraries**

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize 
import folium 

**ID & SECRET Foursquare**

In [3]:
CLIENT_ID = '2Y3IB5KEL5ER4ZUOFMSJLR1ABIHBEKYW2XWUXN5NSY1Y0CCH' 
CLIENT_SECRET = 'OD4GQHVFETRL0V5OCMV4EVVPQMMZP0LZ50UCXQ45AKMN051V'
VERSION = '20180605'

In [4]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") 
    results[city] = requests.get(url).json()

In [5]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

**The FoursquareAPI Only gives us the nearest 100 venues in the city**

*Let's check out their densities by our eyes*

In [11]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  291
Showing Top 100
Total number of pizza places in Chicago, IL =  225
Showing Top 100
Total number of pizza places in San Francisco, CA =  168
Showing Top 100
Total number of pizza places in Jersey City, NJ =  125
Showing Top 100
Total number of pizza places in Boston, MA =  188
Showing Top 100


**map of pizza places in New York**

In [8]:
maps[cities[0]]

**map of pizza places in Chicago**

In [13]:
maps[cities[1]]

**map of pizza places in San Francisco**

In [14]:
maps[cities[2]]

**map of pizza places in Jersey City**

In [15]:
maps[cities[3]]

**map of pizza places in Boston**

In [16]:
maps[cities[4]]

We can see that New York and Jersey City are the most dense cities with Pizza places. And better than that, they are just one shore away.

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [18]:
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02215841512929573
Chicago, IL
Mean Distance from Mean coordinates
0.05717116865688685
San Francisco, CA
Mean Distance from Mean coordinates
0.02903900688689431
Jersey City, NJ
Mean Distance from Mean coordinates
0.019934524532142645
Boston, MA
Mean Distance from Mean coordinates
0.03582491356258987


In [19]:
maps[cities[0]]

In [20]:
maps[cities[1]]

In [21]:
maps[cities[2]]

In [22]:
maps[cities[3]]

In [23]:
maps[cities[4]]

In [24]:
city = 'Jersey City, NJ'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))

Jersey City, NJ
Mean Distance from Mean coordinates
0.0195457213125996
